# Hands-On: Räumliche Analyse

### **Brauereien & Klöster**, *These: Brauereien sind häufig in der Nähe von Klöstern entstanden.*

Im Ordner `data` finden sich Daten zur Lage von Klöstern (`monastaries.csv`) und Brauerein (`breweries.csv`). In dieser Übung, werden wir diese Daten zunächst auf einer Karte darstellen und dann für jede Brauerei, das nächstgelegene Kloster ermitteln.

Wir benötigen drei Pakete:

- `pandas` -> Tabellenkalkulation in Python. In `pandas` wird eine Tabelle als `DataFrame` bezeichnet.
- `shapely` -> beinhaltet Datentypen für Geometrien (z.B. Punkte, Linien, Polygone)
- `geopandas` -> Erweiterung von `pandas`. Assoziert jede Tabellenzeile mit einer `shapely`-Geometrie. Ermöglicht z.B. räumliche Abfragen: Welche Zeilen befinden sich innerhalb eines bestimmten Bereichs?

In einem GeoPandas `GeoDataFrame` repräsentiert jede Zeile ein räumliches Objekt (-> Feature). Jedes Objekt hat eine ID, eine beliebige Anzahl weiterer Eigenschaften (Attribute) sowie eine räumliche Definition (automatisch angelegte Spalte `geometry`), siehe https://geopandas.org/en/stable/getting_started/introduction.html.

<img src="../assets/geodataframe.png" width=450px>


In [ ]:
# imports
import pandas
import shapely
import geopandas

### Einlesen der Klöster- & Brauereitabellen

1. Einlesen als `pandas`-Data Frame mit der `pandas`-Funktion [`read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)
    - `converters` -> Erlaubt die Definition von Funktionen für das Einlesen bestimmter Spalten. `shapely.wkt.loads` interpretiert einen String als `shapely`-Geometrie
    - `dtype` -> Erlaubt die Zuweisung elementarer Datentypen für bestimmte Spalten. Standard für numerische Strings sind Gleitkommazahlen, Jahre sind aber ganzzahlig.
2. Konvertierung in einen `geopandas` GeoDataFrame & Definition des Koordinatensystems (CRS = Coordinate Reference System). `4283` ist der Identifikator für [World Geodetic System 1984](https://epsg.io/4326) -> Kugelkoordinaten, Angabe in Grad (Breitengrad, Längengrad | Latitude, Longitude | lat, lon)

In [ ]:
monasteries = pandas.read_csv('../data/monasteries.csv', converters={'geometry': shapely.wkt.loads}, dtype={'foundingYear': int})
monasteries = geopandas.GeoDataFrame(monasteries, crs=4326)
monasteries.head()

In [ ]:
# data type of entry in the 1st row of column 'geometry'
type(monasteries['geometry'][0])

In der Brauereitabelle ist nicht für jede Brauerei ein Gründungsjahr angegeben. Darum definieren wir eine Funktion, die das Gründungsjahr nur dann als einen Ganzzahl interpretiert, wenn auch ein Wert vorhanden ist. Die Verwendung von `dtype` würde einen Fehler verursachen, wenn in einer Zeile kein Wert für Gründungsjahr vorhanden ist, da ein nicht vorhandener Wert nicht als Ganzahl interpretiert werden kann.

In [ ]:
breweries = pandas.read_csv('../data/breweries.csv', converters={'geometry': shapely.wkt.loads, 'foundingYear': lambda x: int(x) if x else x})
breweries = geopandas.GeoDataFrame(breweries, crs=4326)
breweries.head()

Die Funktion [`explore()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html) von `geopandas` stellt eine GeoDataFrame auf einer Karte dar.

In [ ]:
monasteries.explore(
    color = "cornflowerblue", 
    marker_kwds=dict(radius=5),
    popup =True,    
    width =800,
    height = 500
    )


Zur Darstellung mehrerer GeoDataFrames auf einer Karte, kann der Rückgabewert von `explore()` (also die Karte) in einer Variable gespeichert werden (`map`). Die Variable kann dann, wenn `explore()` für einen anderen GeoDataFrame aufgerufen wird, übergeben werden (`m = map`).

In [ ]:
map = monasteries.explore(
    color = "cornflowerblue", 
    marker_kwds=dict(radius=5), 
    popup =True,
    width =800,
    height = 500
    )
breweries.explore(
    m = map, 
    color = "tomato", 
    marker_kwds=dict(radius=3), 
    popup =True
    )

[`sjoin_nearest`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.sjoin_nearest.html#geopandas.GeoDataFrame.sjoin_nearest) steht für *spatial join nearest*, also die Verknüpfung zweier GeoDataFrames auf Basis der geringsten Distanz. Für jede Zeile im linken DataFrame (`left_df`), wird der rechte DataFrame (`right_df`) nach dem Eintrag mit der geringesten räumlichen Distanz durchsucht. Der Eintrag mit der geringesten Distanz wird dieser Zeile dann hinzugefügt, die Distanz wird in der Spalte "distance" gespeichert (definiert mit `distance_col="distance"`).

In [ ]:
joined_df = geopandas.sjoin_nearest(left_df = breweries, right_df=monasteries, distance_col="distance")
joined_df.head()

Warnung: `UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.`

Ein __geographisches Koordinatensystem__, ist ein System das auf einer __spährischen Oberfläche__ basiert. Die Angabe der Koordinaten erfolgt in Grad.<br>
Ein __projiziertes Koordinatensystem__, projiziert eine spährische Oberfläche auf eine __Ebene__. 

Die Funktion `sjoin_nearest()`, nutzt den Satz des Pythagoras zur Distanzbestimmung (nur in der Ebene gültig). Es gibt sehr viele projizierte Koordinatensysteme, die sich, je nach Anwendungsfall, besser oder schlechter eignen. 

Frage an ChatGPT: _„What is a suitable projected CRS for Germany?“_

>*“There are several projected coordinate reference systems (CRS) that are suitable for use in Germany, depending on the scale and purpose of your analysis. Here are a few options:*
> 
> *[...]*
>
> *__2. ETRS89 / UTM Zone 32N (EPSG:25832)__: This is a commonly used projected CRS for Germany that is based on the European Terrestrial Reference System 1989 (ETRS89) and the UTM 32N zone. It has units of meters and is suitable for local-scale analyses.“*

Die Geometrien eines GeoDataFrames können mit der `geopandas`-Funktion `to_crs()` in ein anderes Koordinatensystem projiziert werden.

In [ ]:
breweries_projected = breweries.to_crs("EPSG:25832")
monasteries_projected = monasteries.to_crs("EPSG:25832")

Erneuter Spatial Join. Dieses Mal geben wir die Einheit der Distanz mit im Spaltennamen an.

In [ ]:
joined_df = geopandas.sjoin_nearest(left_df = breweries_projected, right_df=monasteries_projected, distance_col="distance[m]")
joined_df.head()

Einstellung eines Histogramms mit 100 Körben für die Verteilung der Werte in der Spalte `distance[m]`.

In [ ]:
joined_df['distance[m]'].plot.hist(bins=100)

Im nächsten Codeblock wird der GeoDataFrame etwas aufgeräumt. Wir ordnen die Spalten neu an, bennenen einige um und sortieren die Einträge nach der Spalte `distance[m]`.

In [ ]:
# reorder columns
joined_df = joined_df.reindex(columns=['brewery', 'monastery', 'distance[m]', 'foundingYear_left', 'foundingYear_right', 'geometry'])
# rename founding year columns
joined_df = joined_df.rename(columns={'foundingYear_left': 'founding_brewery', 'foundingYear_right': 'founding_monastery'})
# sort, reindex and drop old index
joined_df = joined_df.sort_values(by='distance[m]').reset_index().drop(columns='index')
joined_df.head(n=20)

Erneutes Darstellen des verknüpften GeoDataFrames. Mit der Angabe `column='distance[m]'` geben wir der Funktion `explore()` zu verstehen, dass wir uns für diese Spalte interessieren. Dies veranlasst, die Funktion dazu, die Marker, entspechend der Werte dieser Spalte, farbig zu kodieren. Mit Hilfe der `style_function` skalieren wir den Radius der Marker zuzätzlich in Abhängigkeit der Distanz (Die Größe der Marker entspricht jedoch nicht der tatsächlichen Distanz!).

In [ ]:
joined_df.explore(
    column='distance[m]', 
    style_kwds={"style_function": lambda x: {"radius": x["properties"]["distance[m]"]/1000}}, 
    popup =True,    
    width =800,
    height = 500
    )


Da wird unseren GeoDataFrame aufsteigend, nach der Distanz zum nächsten Kloster, sortiert haben, werden die Marker mit kleinem Radius, vor denen mit großem Radius gezeichnet. Wir sortieren den Datensatz nach absteigender Distanz, definieren eine Mindestgröße der Kreise (`+5` am Ende der `style_funtion`) und wählen eine andere Farbkodierung (`cmap = "cividis_r"`). 

- [Verfügbare Colormaps](https://matplotlib.org/1.2.1/mpl_examples/pylab_examples/show_colormaps.pdf)

In [ ]:
joined_df = joined_df.sort_values(by='distance[m]', ascending=False).reset_index().drop(columns='index')
joined_df.explore(
    column='distance[m]', 
    style_kwds={"style_function": lambda x: {"radius": (x["properties"]["distance[m]"]/1000)+5}},  
    popup =True,
    width =800,
    height = 500,
    cmap = "winter_r"
    )
